# <center>MO444 - Aprendizado de Máquina e Reconhecimento de Padrões</center>
## <center>Primeiro semestre de 2021</center>
## <center>Projeto 3: Algoritmos Evolucionários e Aprendizado por Reforço</center>
## <center>Discentes: Elisa Dell'Arriva (135551) e Felipe de Carvalho Pereira (230214)</center>
_________________________________________________________________________

# 1 Introdução

O presente *notebook* consiste no terceiro projeto avaliativo da disciplina MO444 - Aprendizado de Máquina e Reconhecimento de Padrões, ministrada pela Profa. Dra. Esther Colombini. O principal objetivo da atividade foi o de implementar e experimentar técnicas de Algoritmos Evolucionários e de Aprendizado por Reforço para resolver o Problema do Pacman.<br>

Na primeira parte do projeto, descrita na Seção 2, buscamos resolver o Problema do Pacman por meio de um algoritmo de Programação Genética baseado nos métodos empregados em [4] e [5]. Na segunda etapa, descrita na Seção 3, lidamos com o mesmo problema, mas empregamos uma técnica de Aprendizado por Reforço denoniminada Q-learning, de maneira semelhante ao que é apresentado em [6]. Na Seção 4, realizamos um experimento adicional a fim de comparar os algoritmos desenvolvidos nas Seções anteriores.

## 1.1 O Problema do Pacman

O famoso jogo do Pacman se passa em um labirinto, na forma de um tabuleiro, em que o Pacman, o agente controlado pelo usuário, movimenta-se por meio das células (ou casas). O objetivo do jogo é coletar todos os itens de comida dispostos no labirinto. Além do Pacman, existem outros agentes que se movimentam pelo labirinto. Estes são denominados fantasmas e não são controlados pelo usuário. O jogo acaba somente em duas ocasiões: quando todos os itens de comida forem consumidos e, neste caso, decreta-se vitória; ou quando o Pacman colide com algum fantasma, e neste caso, decreta-se derrota.

Além dos itens de comida, existem cápsulas que também são consumíveis pelo Pacman. No momento em que uma cápsula é consumida, a velocidade com a qual os fantasmas se movimentam é reduzida, e os mesmos tornam-se comestíveis pelo Pacman. Contudo, tais efeitos persistem por um curto intervalo de tempo. Quando um fantasma está sob o efeito da cápsula, dizemos que ele está *comestível*, caso contrário dizemos que ele está *ativo*. A pontuação do usuário durante o jogo é baseada nas seguintes regras:

* A pontuação no início do jogo é igual a 0;
* A cada iteração do jogo, a pontuação é decrementada em uma unidade;
* Quando um item de comida é consumido, a pontuação é incrementada em 10 unidades;
* Quando um fantasma comestível é consumido, a pontuação é incrementada em 200 unidades;
* Quando o Pacman colide com um fantasma ativo a pontuação é decrementada em 500 unidades;
* Quando o Pacman consome o último item de comida, a pontuação é incrementada em 500 unidades.

Nesse contexto, o Problema do Pacman corresponde em desenvolver um agente capaz de controlar o Pacman com o objetivo de vencer o jogo e maximizar a pontuação obtida.

Junto à especificação deste projeto, foi fornecida uma API que implementa o jogo do Pacman na linguagem Python 2. Contudo, uma versão em Python 3 desta mesma API está disponível em [7] e esta última foi aquela empregada neste trabalho. Em particular, os mapas (ou *layouts*) utilizados foram: smallClassic, mediumClassic e originalClassic.

# 2 Parte $I$ - Modelo Evolucionário

Nesta Seção, descrevemos a abordagem baseada em modelos evolucionários utilizada neste trabalho para resolver o Problema do Pacman.

## 2.1 Algoritmos Genéticos

Algoritmos genéticos são abordagens heurísticas nas quais modelam-se soluções de um determinado problema como indivíduos de uma população. Em cada iteração do algoritmo, tem-se uma geração (no sentido temporal) da população. Ao longo das gerações, os indivíduos se reproduzem e sofrem mutações, de modo análogo ao que é observado na natureza com diversas espécies. Em particular, o algoritmo aplica um procedimento semelhante ao processo de Seleção Natural, de tal modo que apenas os indivíduos mais adaptados possam sobreviver através das gerações.

A maneira como soluções são representadas por indivíduos varia de acordo com o problema tratado. Além disso, a literatura (vide [1,2,3]) apresenta diversas maneiras de se implementar as etapas de reprodução (*crossover*), mutação e substituição de população. Em linhas gerais, um algoritmo genético funciona da seguinte maneira.

* 1 - Uma população inicial de indivíduos (possivelmente produzidos de maneira aleatorizada) é estabelecida;
* 2 - Pares de indivíduos (ou pais) são selecionados para reprodução e novos indivíduos (filhos) são produzidos, os quais herdam características de ambos os pais;
* 3 - Uma fração da população é escolhida para sofrer mutações, isto é, alterações randomizadas em sua estrutura;
* 4 - Cada indivíduo é avaliado de acordo com uma fução de aptidão (ou *fitness*);
* 5 - Os indivíduos considerados mais aptos são escolhidos para compor a próxima geração da população;
* 6 - Repete-se os passos 2 a 5 até que a população convirja ou que o limite de iterações preestabelecido seja atingido.

## 2.2 Programação Genética

Neste trabalho, escolhemos a técnica denominada Programação Genética para resolver o Problema do Pacman. A Progração Genética pode ser entendida como uma variação dos algoritmos genéticos em que cada indivíduo da população corresponde a um programa (*software*) ao invés de uma solução propriamente dita. Quando executado, o programa (indivíduo) deve ser capaz de produzir uma solução para o problema.

No contexto do jogo do Pacman, um indivíduo consiste em uma árvore de decisão por meio da qual chega-se a uma escolha de movimentação do Pacman, considerando o estado corrente do jogo. Denominaremos esse tipo de árvore de decisão como *política*, i.e., uma política de movimentação do Pacman. Para cada iteração do jogo tem-se as informações a respeito da disposição dos itens e agentes no labirinto. Assim, quando uma política é executada, a mesma se utiliza das informações do labirinto para decidir o próximo movimento do Pacman (norte, sul, leste, oeste ou ficar parado). 

A seguir, descrevemos a modelagem de Programação Genética que empregamos para resolver o Problema do Pacman, a qual foi baseada nos algoritmos desenvolvidos em [4,5].

### 2.2.1 Modelagem do indivíduo

Cada indivíduo da população corresponde a uma política e é implementado por meio de árvores (estrutura de dados). Cada nó da árvore tem uma única natureza entre as que são descritas abaixo:

* Data: valor numérico que pode ser uma constante ou uma informação acerca do atual estado do tabuleiros;
* Action: representa o objetivo que baseará a próxima ação do Pacman;
* Iflte: (*If less then else*): nó que possui quatro filhos, sendo os dois primeiros da natureza Data e os últimos da natureza Action ou Iflte.

Se um nó é do tipo Data, então ele tem alguma das seguintes subnaturezas:

* DistToFood: a menor das distâncias de Manhattan entre a posição do Pacman e as posições dos itens de comida restantes;
* DistToCapsule: a menor das distâncias de Manhattan entre a posição do Pacman e as posições das cápsulas restantes;
* DistToActiveGhost: a menor das distâncias de Manhattan entre a posição do Pacman e as posições dos fantasmas ativos;
* DistToEdibleGhost: a menor das distâncias de Manhattan entre a posição do Pacman e as posições dos fantasmas comestíveis;
* Constant: um valor constante entre 1 e o *maxConst*, onde *maxConst* é um parâmetro.

Se um nó é do tipo Action, então ele tem alguma das seguintes subnaturezas:

* GetFood: Pacman deve se movimentar em direção ao item de comida mais próximo;
* GetCapsule: Pacman deve se movimentar em direção à capsula mais mais próxima;
* EscapeFromGhost: Pacman deve se movimentar na direção que o deixa mais distintante do fantasma mais próximo;
* GetEdibleGhost: Pacman deve se movimentar em direção à capsula mais mais próxima.

Apenas nós do tipo Iflte possuem filhos e, em particular, a raiz da árvore de decisão é sempre do tipo Iflte.

### 2.2.2 *Parsing* do indivíduo

Por meio da API do Pacman, criamos uma classe de agente que implementa (sobrescreve) a função *getAction*. A função *getAction* recebe o objeto *state* e deve retornar a ação que o Pacman irá realizar, i.e., ir para a posição adjacente ao norte, sul, leste ou oeste ou ainda permanecer na posição atual. O objeto *state* é útil para coletar informações acerca do atual estado do labirinto. Observe que durante uma partida do jogo, a função *getAction* é invocada a cada iteração do jogo.

Em nossa modelagem, a função *getAction* invoca uma função de *parsing* que recebe o objeto *state* e uma política. Na função de *parsing*, a árvore de decisão (política) é percorrida a partir da raiz até algum nó folha que é uma ação. A ação determina um movimento do Pacman que é retornado pelo *parser* e, em seguida retornado pela função *getAction*.

A seguir, detalharemos a função de *parsing*. Primeiramente, visitamos o nó raiz da política. Como já mencionamos, trata-se de um nó do tipo Iflte. Para esse tipo, faz-se o que segue. Primeiro avaliamos o valor numérico dos dois primeiros filhos do nó raiz, os quais são do tipo Data. Se o valor do primeiro filho for menor que o do segundo, então o terceiro filho do nó raiz é o próximo a ser visitado, caso contrário, o quarto filho é o próximo a ser visitado. Como vimos, esses dois últimos só podem ser do tipo Action ou Iflte. Se o próximo nó for do tipo Iflte, repete-se o procedimento descrito anteriormente. Caso contrário, se for do tipo Action, então é executado um procedimento que determina a ação do Pacman e o *parsing* é encerrado.

Quando o valor de um nó do tipo Data é avaliado, faz-se o que segue de acordo com a subnatureza do nó. Se a subnatureza é Constant, então já existe um valor numérico constante associado àquele nó, o qual é retornado. Caso contrário, trata-se de uma subnatureza que é a distância do Pacman para objetos do mesmo tipo: comida, cápsula, fantasma ativo ou comestível. Nesse caso, calcula-se a distância (de Manhattan) entre o Pacman e todos os objetos do daquele tipo e retorna-se a menor das distâncias. Por exemplo, se a subnatureza é DistToFood, então é retornada a distância (de Manhattan) entre a posição do Pacman e a posição do item de comida mais próximo (considerando a distância de Manhattan).

Quando um nó do tipo Ação é visitado, faz-se o que segue de acordo com a subnatureza do nó. Se a subnatureza é EscapeFromGhost, então calculamos a distância (de Manhattan) entre a posição do Pacman e o fantasma ativo mais próximo (considerando a distância de Manhattan). Em seguida, escolhemos a movimentação do Pacman que aumenta ou mantém a distância em relação àquele fantasma. Na prática isso significa fugir daquele fantasma com uma escolha gulosa de movimentação. Se a subnatureza é diferente de EscapeFromGhost, trata-se de uma subnatureza em que o Pacman objetivará comer algum item: comida, cápsula ou fantasma comestível. Nesse caso, calcula-se o caminho mais curto no labirinto entre o Pacman e o item mais próximo. O movimento do Pacman será aquele que vai em direção ao item nesse caminho. Para calcular esses caminhos, implementamos uma estrutura de dados adicional em que as células acessíveis do labirinto são vértices de um grafo e, dessa forma, a menor caminho entre duas células é calculado por uma busca em largura naquele grafo. Observe que sempre que um menor caminho entre duas células é calculado, o armazenamos em memória para não que não seja necessário recalculá-lo, no caso em que o cenário venha a se repetir. No caso especial em que não restam itens do tipo pelo qual ação objetiva alcançar, a ação padrão tomada é GetFood.

### 2.2.3 Operações genéticas

Adiante, descrevemos as operações relativas a evolução da população, tais como *crossover*, mutação e substituição, que são adotadas no nosso modelo. A população inicial é produzida com indivíduos gerados de maneira aleatorizada com profundidade da árvore de decisão limitada a um parâmetro preestabelecido. Além disso, o tamanho $n$ da população também é parametrizado, porém é fixo para todas as iterações do algoritmo.

Seja $P$ o conjunto de $n$ indivíduos da $i$-ésima geração. O primeiro passo é fazer com que os indivíduos se reproduzam. Para isso, selecionamos dois indivíduos $p_1$ e $p_2$ de $P$ de maneira puramente aleatória. Em seguida, criamos uma cópia $f_1$ de $p_1$ e uma cópia $f_2$ de $p_2$. Depois, elencamos uma lista de nós do tipo Iflte (excetuando o nó raiz) de $f_1$ e fazemos o mesmo para $f_2$. Mais à frente, sorteamos de tais listas (de maneira puramente aleatória) um nó Iflte de $f_1$ e um nó Iflte de $f_2$ e trocamos, entre $f_1$ e $f_2$, as subárvores enraizadas nesses nós. Por fim acrescentamos $f_1$ e $f_2$ a população $P$. Repetimos esse procedimento até que o total de filhos gerados atinja um valor parametrizado.

Em seguida, aplicamos o processo de mutação. Para isso, selecionamos um subconjunto $X$ de $P$ de indivíduos de $P$ e para cada $x \in X$, criamos uma cópia $x'$ de $x$ e alteramos $x$ da seguinte maneira. Elencamos uma lista de nós de $x'$ que não são do tipo Iflte e sorteamos um deles. Se o nó for do tipo Action, alteramos a subnatureza do nó para uma subnatureza diferente e de maneira aleatorizada. Se o nó for do tipo Data, temos dois casos. Se a subnatureza for Constant, então alteramos aleatoriamente o valor da constante associada ao nó. Caso contrário, alteramos a subnatureza do nó para alguma que é diferente da atual e diferente de Constant. Em seguida adicionamos $x$ a $P$.

O penúltimo passo é avaliar o *fitness* de cada indivíduo de $P$. Antes de descrever a função de *fitness*, precisamos descrever o que consideramos como pontuação final de um jogo. Em termos de vitória ou derrota, mantemos a pontuação original do jogo: o Pacman vence se comer todos os itens de comida e perde se colidir com algum fantasma ativo. Contudo, alteramos o cálculo a pontuação propriamente dita. Em caso de derrota, a pontuação final é aquela retornada pela API. Em caso de vitória, somamos 10.000 pontos à pontuação retornada pela API. A razão dessa adoção é baseada no fato de que, se mativermos a pontuação original, em muitos casos é possível que o agente tenha sido derrotado e termine com uma pontuação maior do que se tivesse vencido. Assim, os 10.000 pontos adicionais servem para termos uma maior discrepância na pontuação entre os casos de vitória e derrota. Ademais, no nosso modelo, o *fitness* de um indivíduo é igual à média das pontuações de 10 execuções do jogo com sementes (de geração de número pseudoaleatório) distintas.

Por fim, os $n$ indivíduos de $P$ com maior *fitness* são escolhidos para constituir a $i+1$-ésima geração.

## 2.3 Código-Fonte

A seguir, temos o código-fonte relativo à Parte I do projeto.

In [1]:
# Importação de módulos
from pacman import *
import numpy as np
import random
import math
from game import Directions
from game import Agent
import copy
import math

###############################################################################
# Implementações relativas à obtenção de caminhos entre células do tabuleiro. #
###############################################################################

# Classe que representa uma célula do labirinto
class Cell:
    coordinates = None
    adjacency = np.array([])
    visited = False
    parent = None

# Função que retorna lista de células do grafo que representa o labirinto
# e dicionário de que mapeia as coordenadas das células nos objetos que são células.
def get_list_and_dict_of_cells(layout):
    dict_of_cells = dict([])
    list_of_cells = np.array([])

    for x in range(layout.width):
        for y in range(layout.height):
            if not layout.walls[x][y]:
                new_cell = Cell()
                new_cell.coordinates = (x,y)
                dict_of_cells[str(x) + ',' + str(y)] = new_cell
                list_of_cells = np.append(list_of_cells, new_cell)

    for cell in list_of_cells:
        x = cell.coordinates[0]
        y = cell.coordinates[1]

        if x - 1 >= 0 and not layout.walls[x-1][y]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x-1) + ',' + str(y)])

        if y - 1 >= 0 and not layout.walls[x][y-1]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x) + ',' + str(y-1)])

        if x + 1 <= layout.width and not layout.walls[x+1][y]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x+1) + ',' + str(y)])

        if y + 1 <= layout.height and not layout.walls[x][y+1]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x) + ',' + str(y+1)])
    
    return list_of_cells, dict_of_cells
    
# Função que executa uma busca em profundidade no grafo que representa o labirinto
# e determina o menor caminho no labirinto entre essas duas células.
def find_shortest_path(initial_cell_coordinates, final_cell_coordinates, list_of_cells, dict_of_cells):
    
    initial_cell = dict_of_cells[str(initial_cell_coordinates[0]) + ',' + str(initial_cell_coordinates[1])]
    final_cell = dict_of_cells[str(final_cell_coordinates[0]) + ',' + str(final_cell_coordinates[1])]
    
    for cell in list_of_cells:
        cell.visited = False
        cell.parent = None
    
    queue = []
    queue.append(initial_cell)
    
    while not final_cell.visited:
        current_cell = queue.pop(0)
        current_cell.visited = True
        for adjacent_cell in current_cell.adjacency:
            if not adjacent_cell.visited:
                queue.append(adjacent_cell)
                adjacent_cell.parent = current_cell
                
    path = []
    current_cell = final_cell
    while current_cell != initial_cell:
        path.append(current_cell)
        current_cell = current_cell.parent
    path.append(current_cell)
    
    path.reverse()
    
    for i in range(len(path)-1):
        direction = None
        if path[i].coordinates[0] > path[i+1].coordinates[0]:
            direction = Directions.WEST
        elif path[i].coordinates[0] < path[i+1].coordinates[0]:
            direction = Directions.EAST
        elif path[i].coordinates[1] > path[i+1].coordinates[1]:
            direction = Directions.SOUTH
        else:
            direction = Directions.NORTH
        key = str(path[i].coordinates[0]) + ',' + str(path[i].coordinates[1]) + '-' + str(final_cell.coordinates[0]) + ',' + str(final_cell.coordinates[1])
        greedy_moves[key] = direction


#######################################################################
# Implementações relativas ao agente do Pacman (Programação Genética) #
#######################################################################

# Listas de naturezas e subnaturezas
node_natures = ['Iflte', 'Action', 'Data']
data_subnatures = ['DistToFood', 'DistToCapsule', 'DistToActiveGhost', 'DistToEdibleGhost', 'Constant']
action_subnatures = ['GetFood', 'GetCapsule', 'EscapeFromGhost', 'GetEdibleGhost']

# Classe que representa um nó de uma árvore de decisão,
# i.e. o nó de uma política, ou ainda, o nó de um indivíduo.
class Node:
    isRoot = False
    nature = None
    subnature = None
    value = None
    parent = None
    children = np.array([])
    fitness = None
    win_counter = None
    
# Classe que implementa o agente.
# O único atributo, policy, deve receber o a árvore de decisão.
class GpAgent(Agent):

    # Recebe a árvore de decisão (política).
    policy = None
    
    # Função que encontra o item de comida mais próximo do Pacman.
    def find_closest_food(self, state):
        food_list = state.getFood().asList()
        closest_food_coordinates = None
        closest_food_dist = math.inf
        for i in range(len(food_list)):
            current_dist = manhattanDistance(state.getPacmanPosition(), food_list[i])
            if current_dist < closest_food_dist:
                closest_food_dist = current_dist
                closest_food_coordinates = food_list[i]
        return closest_food_coordinates, closest_food_dist
    
    # Função que encontra a cápsula mais próxima do Pacman.
    def find_closest_capsule(self, state):
        capsule_list = state.getCapsules()
        closest_capsule_coordinates = None
        closest_capsule_dist = math.inf
        for i in range(len(capsule_list)):
            current_dist = manhattanDistance(state.getPacmanPosition(), capsule_list[i])
            if current_dist < closest_capsule_dist:
                closest_capsule_dist = current_dist
                closest_capsule_coordinates = capsule_list[i]
        return closest_capsule_coordinates, closest_capsule_dist
    
    # Função que encontra o fantasma ativo mais próximo do Pacman.
    def find_closest_active_ghost(self, state):
        closest_ghost_coordinates = None
        closest_ghost_dist = math.inf
        for i in range(1, state.getNumAgents()):
            if state.data.agentStates[i].scaredTimer <= 1:
                ghost_position = state.getGhostPosition(i)
                ghost_position = (int(ghost_position[0]), int(ghost_position[1]))
                current_dist = manhattanDistance(state.getPacmanPosition(), ghost_position)
                if current_dist < closest_ghost_dist:
                    closest_ghost_dist = current_dist
                    closest_ghost_coordinates = ghost_position
        return closest_ghost_coordinates, closest_ghost_dist
    
    # Função que encontra o fantasma comestível mais próximo do Pacman.
    def find_closest_edible_ghost(self, state):
        closest_ghost_coordinates = None
        closest_ghost_dist = math.inf
        for i in range(1, state.getNumAgents()):
            if state.data.agentStates[i].scaredTimer > 1:
                ghost_position = state.getGhostPosition(i)
                ghost_position = (int(ghost_position[0]), int(ghost_position[1]))
                current_dist = manhattanDistance(state.getPacmanPosition(), ghost_position)
                if current_dist < closest_ghost_dist:
                    closest_ghost_dist = current_dist
                    closest_ghost_coordinates = ghost_position
        return closest_ghost_coordinates, closest_ghost_dist    
    
    
    # Função que faz o parsing da política carregada no atributo policy.
    def parse_policy(self, policy, state): 
        
        current_node = policy # Carrega a raiz da árvore de decisão 
        
        # Percorre a árvore de acordo com a descrição feita na Seção 2.2.2
        if current_node.nature == 'Iflte':
            children = current_node.children
            for child in children[:2]:
                if child.subnature == 'DistToFood':
                    child.value = self.find_closest_food(state)[1]
                elif child.subnature == 'DistToCapsule':
                    child.value = self.find_closest_capsule(state)[1]
                elif child.subnature == 'DistToActiveGhost':
                    child.value = self.find_closest_active_ghost(state)[1]
                elif child.subnature == 'DistToEdibleGhost':
                    child.value = self.find_closest_edible_ghost(state)[1]
            if children[0].value < children[1].value:
                return self.parse_policy(children[2], state)
            else:
                return self.parse_policy(children[3], state)
        elif current_node.nature == 'Action':
            subnature = current_node.subnature 
            if subnature == 'GetCapsule':
                closest_capsule_coordinates = self.find_closest_capsule(state)[0]
                if closest_capsule_coordinates != None:
                    current_position = list(state.getPacmanPosition())
                    key = str(current_position[0]) + ',' + str(current_position[1]) + '-' + str(closest_capsule_coordinates[0]) + ',' + str(closest_capsule_coordinates[1])
                    if key not in greedy_moves:
                        find_shortest_path(current_position, closest_capsule_coordinates, list_of_cells, dict_of_cells)
                    action = greedy_moves[key]
                    return action
                else:
                    subnature = 'GetFood'
                 
            if subnature == 'GetEdibleGhost':
                closest_edible_ghost_coordinates = self.find_closest_edible_ghost(state)[0]
                if closest_edible_ghost_coordinates != None:
                    current_position = list(state.getPacmanPosition())
                    key = str(current_position[0]) + ',' + str(current_position[1]) + '-' + str(closest_edible_ghost_coordinates[0]) + ',' + str(closest_edible_ghost_coordinates[1])
                    if key not in greedy_moves:
                        find_shortest_path(current_position, closest_edible_ghost_coordinates, list_of_cells, dict_of_cells)
                    action = greedy_moves[key]
                    return action
                else:
                    subnature = 'GetFood'
                
            if subnature == 'GetFood':
                closest_food_coordinates = self.find_closest_food(state)[0]
                current_position = list(state.getPacmanPosition())
                key = str(current_position[0]) + ',' + str(current_position[1]) + '-' + str(closest_food_coordinates[0]) + ',' + str(closest_food_coordinates[1])
                if key not in greedy_moves:
                    find_shortest_path(current_position, closest_food_coordinates, list_of_cells, dict_of_cells)
                action = greedy_moves[key]
                return action
            if subnature == 'EscapeFromGhost':
                current_position = list(state.getPacmanPosition())
                closest_ghost_coordinates = self.find_closest_active_ghost(state)[0]
                if closest_ghost_coordinates != None:
                    max_dist = 0
                    max_dist_move = None
                    for action in state.getLegalPacmanActions():
                        if action == Directions.WEST:
                            next_position = [current_position[0] - 1, current_position[1]]
                        elif action == Directions.EAST:
                            next_position = [current_position[0] + 1, current_position[1]]
                        elif action == Directions.NORTH:
                            next_position = [current_position[0], current_position[1] + 1]
                        elif action == Directions.SOUTH:
                            next_position = [current_position[0], current_position[1] - 1]
                        else:
                            next_position = [current_position[0], current_position[1]]
                        if manhattanDistance(next_position, closest_ghost_coordinates) > max_dist:
                            max_dist_move = action
                            max_dist = manhattanDistance(next_position, closest_ghost_coordinates)
                    action = max_dist_move
                else:
                    action = Directions.STOP
                
                return action
    
    # Função que retorna o movimento do Pacman com base no resultado do parsing.
    def getAction(self, state):
        action = self.parse_policy(self.policy, state)
        return action
    

###################################################
# Implementações relativas ao algoritmo genético. #
###################################################

# Variáveis globais
args = agent, layout, list_of_cells, dict_of_cells, greedy_moves = None

# Imprime uma árvore de decisão.
def print_policy(root):
    if root.value != None:
        print(root.value)
    elif root.subnature != None:
        print(root.subnature)
    else:
        print(root.nature)
    print()
    for c in root.children:
        print_policy(c)

# Constrói uma árvore de decisão (política) de maneira aleatorizada.
def build_random_policy(node, isRoot, max_level, max_const):
    node.nature = 'Iflte'
    if isRoot:
        node.isRoot = True
        
    c1 = Node()
    c2 = Node()
    c3 = Node()
    c4 = Node()
    
    for c in [c1,c2,c3,c4]:
        c.parent = node

    if node.isRoot:
        c1.nature = 'Data'
        c1.subnature = 'DistToActiveGhost'
        c2.nature = 'Data'
        c2.subnature = 'Constant'
        c2.value = random.randint(1, max_const)
        c3.nature = 'Action'
        c3.subnature = 'EscapeFromGhost'
        c4.nature = 'Iflte'
        build_random_policy(c4, False, max_level-1, max_const)  
    else:
        c1.nature = 'Data'
        c1.subnature = random.choice([item for item in data_subnatures if item != 'Constant'])
        c2.nature = 'Data'  
        c2.subnature = random.choice([item for item in data_subnatures if item != c1.subnature])
        if c2.subnature == 'Constant':
            c2.value = random.randint(1, max_const)
        if max_level > 1:
            c3.nature = random.choice(['Action', 'Iflte'])
            c4.nature = random.choice(['Action', 'Iflte'])
        else:
            c3.nature = 'Action'
            c4.nature = 'Action'

        if c3.nature == 'Action':
            c3.subnature = random.choice(action_subnatures)
        elif c3.nature == 'Iflte':
            build_random_policy(c3, False, max_level-1, max_const)

        if c4.nature == 'Action':
            c4.subnature = random.choice([item for item in action_subnatures if item != c3.subnature])
        elif c4.nature == 'Iflte':
            build_random_policy(c4, False, max_level-1, max_const)
    
    node.children = np.array([c1, c2, c3, c4])

# Função que retorna uma população com indivíduos produzidos aleatoriamente
def get_new_population(population_size, max_depth, max_const):
    population = []

    for i in range(population_size):
        root = Node()
        build_random_policy(root, False, max_depth, max_const)
        population.append(root)

    return np.array(population)

# Função que calcula o fitness de uma política.
def evaluate_policy(policy):
    agent.policy = policy
    win_counter = 0
    fitness = 0
    
    for i in range(10):
        games = runGames(**args)
        fitness += games[0].state.getScore()
        if games[0].state.isWin():
            win_counter += 1
            fitness += 10000
    fitness = float(fitness)/10
    
    policy.fitness = fitness
    policy.win_counter = win_counter

# Função que retorna os nós de natureza Iflte de uma política (exceto a raiz).
def get_iflte_nodes(policy):
    root = policy
    queue = [root]
    iflte_nodes = []
    while len(queue) > 0:
        current_node = queue.pop(0)
        if current_node.nature == 'Iflte':
            if not current_node.isRoot:
                iflte_nodes.append(current_node)
            for c in current_node.children:
                queue.append(c)
    return np.array(iflte_nodes)

# Função que retorna os nós de natureza não Iflte de uma política.
def get_non_iflte_nodes(policy):
    root = policy
    queue = [root]
    non_iflte_nodes = []
    while len(queue) > 0:
        current_node = queue.pop(0)
        if current_node.nature != 'Iflte':
            non_iflte_nodes.append(current_node)
        for c in current_node.children:
            queue.append(c)
    return np.array(non_iflte_nodes)

# Função que substitui a população.
def replace_population(population, new_population_size): 
    population = sorted(population, key=lambda x:x.fitness, reverse=True)
    new_population = []
    for i in range(new_population_size):
        new_population.append(population[i])
    
    return np.array(new_population)

# Função que faz o crossover em uma geração.
def make_crossover(population, n_crossover):
    
    new_policys = []
    for i in range(n_crossover):
        p1 = copy.deepcopy(random.choice(population))
        p2 = copy.deepcopy(random.choice(population))
        
        p1_iflte_nodes = get_iflte_nodes(p1)
        p2_iflte_nodes = get_iflte_nodes(p2)
    
        p1_selected_iflte_node = random.choice(p1_iflte_nodes)
        p2_selected_iflte_node = random.choice(p2_iflte_nodes)
        
        if p1_selected_iflte_node.parent != None:
            if p1_selected_iflte_node.parent.children[2] == p1_selected_iflte_node:
                p1_selected_iflte_node.parent.children[2] = p2_selected_iflte_node
            else:
                p1_selected_iflte_node.parent.children[3] = p2_selected_iflte_node
        
        if p2_selected_iflte_node.parent != None:
            if p2_selected_iflte_node.parent.children[2] == p2_selected_iflte_node:
                p2_selected_iflte_node.parent.children[2] = p1_selected_iflte_node
            else:
                p2_selected_iflte_node.parent.children[3] = p1_selected_iflte_node

        tmp = p1_selected_iflte_node.parent
        p1_selected_iflte_node.parent = p2_selected_iflte_node.parent
        p2_selected_iflte_node.parent = p1_selected_iflte_node.parent
        
        new_policys.append(p1)
        new_policys.append(p2)
        
    for p in new_policys:
        evaluate_policy(p)
        
    return np.array(new_policys)

# Função que faz o processo de mutação de uma geração.
def make_mutation(population, n_mutations, max_const):
    new_policys = []
    for i in range(n_mutations):
        p = copy.deepcopy(random.choice(population))
        p_non_iflte_nodes = get_non_iflte_nodes(p)
        p_seleted_node = random.choice(p_non_iflte_nodes)
        
        if p_seleted_node.nature == 'Action':
            p_seleted_node.subnature = random.choice([item for item in action_subnatures if item != p_seleted_node.subnature])
        elif p_seleted_node.nature == 'Data':
            p_seleted_node.subnature = random.choice([item for item in data_subnatures if item != p_seleted_node.subnature])
            if p_seleted_node.subnature == 'Constant':
                p_seleted_node.value = random.randint(1, max_const)
        
        new_policys.append(p)
    
    for p in new_policys:
        evaluate_policy(p)
    
    return np.array(new_policys)

# Função que implementa o algoritmo genético
def genetic_algorithm(n_iterations, population_size, max_depth, max_const, n_children, n_mutates):
    population = get_new_population(population_size, max_depth, max_const)
    
    for p in population:
        evaluate_policy(p)
    '''
    j = 1
    for p in population:
        print("#" + str(j))
        print_policy(p)
        print()
        j += 1
    '''
        
    for i in range(n_iterations):
        print("Generation #" + str(i+1))
        
        min_win = math.inf
        max_win = -math.inf
        avg_win = 0
        min_fitness = math.inf
        max_fitness = -math.inf
        avg_fitness = 0
        for p in population:
            min_win = min(min_win, p.win_counter)
            max_win = max(max_win, p.win_counter)
            avg_win += p.win_counter
            min_fitness = min(min_fitness, p.fitness)
            max_fitness = max(max_fitness, p.fitness)
            avg_fitness += p.fitness
        avg_win = float(avg_win)/len(population)
        avg_fitness = float(avg_fitness)/len(population)
        
        print("Min Win:", min_win)
        print("Max Win:", max_win)
        print("Avg Win:", avg_win)
        print("Min Fitness:", min_fitness)
        print("Max Fitness:", max_fitness)
        print("Avg Fitness:", avg_fitness)
        print()
        
        if i+1 < n_iterations:
            new_chromossomes_cross = make_crossover(population, int(n_children/2))
            new_chromossomes_mutation = make_mutation(population, n_mutations, max_const)            
            population = replace_population(np.concatenate((population, new_chromossomes_cross, new_chromossomes_mutation)), population_size)

# Função que inicializa o layout do labirinto
def initialize_layout(layout_name):
    input_str = "-q -l " + layout_name
    args = readCommand(input_str.split())
    agent = GpAgent()
    args['pacman'] = agent
    layout = args['layout']
    list_of_cells, dict_of_cells = get_list_and_dict_of_cells(layout)
    greedy_moves = dict([])

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
seed = 36516285968929
random.seed(seed)

initialize_layout('smallClassic')
n_generations = 200
population_size = 10
tree_max_depth = 5
max_constant = 5
n_offspring = 4
n_mutations = 2
genetic_algorithm(n_generations, population_size, tree_max_depth, max_constant, n_offspring, n_mutations)

In [ ]:
initialize_layout('mediumClassic')
n_generations = 200
population_size = 10
tree_max_depth = 5
max_constant = 5
n_offspring = 4
n_mutations = 2
genetic_algorithm(n_generations, population_size, tree_max_depth, max_constant, n_offspring, n_mutations)

In [ ]:
initialize_layout('originalClassic')
n_generations = 200
population_size = 10
tree_max_depth = 5
max_constant = 5
n_offspring = 4
n_mutations = 2
genetic_algorithm(n_generations, population_size, tree_max_depth, max_constant, n_offspring, n_mutations)

# 2.3 Experimentos

# 3 Parte II - Aprendizado por Reforço

# 4 Comparação entre modelos

# 5 *Considerações Finais*

Neste *notebook*, apresentamos as definições dos problemas explorados, os métodos utilizados para resolvê-los, assim como os resultados obtidos, de acordo com a especificação do projeto. Em particular, destacamos os seguintes pontos que foram exibidos no relatório.

Na Seção 2..

* A
* B
* C

Portanto, cumprimos todas as tarefas previstas na especificação do projeto relativa à Parte $I$.

Na Seção 3 ...

* A
* B
* C
 
Assim, cumprimos todas as tarefas previstas na especificação do projeto relativa à Parte $II$.

Além disso, Na Seção 4, discutimos:

* A
* B
* C

Ambos discentes participaram ativamente durante todo o projeto. O aluno Felipe focou os seus esforços na Parte $I$ do projeto, enquanto a aluna Elisa esteve mais envolvida com a Parte $II$. Contudo, as pesquisas, revisões de código-fonte e tomadas de decisão foram sempre feitas em conjunto. 

# 6 Referências Bibliográficas

[1] ALPAYDIN, E. Introduction to Machine Learning. MIT Press, 3rd edition, 2014.

[2] BISHOP, C. M. Pattern Recognition and Machine Learning. Springer. Cambridge, 2007.

[3] MARSLAND, S. Machine Learning: an algorithm perspective. CRC Press, 2nd edition, 2015.

[4] BRANDSTETTER, Matthias F.; AHMADI, Samad. Reactive control of Ms. Pac Man using information retrieval based on genetic programming. In: 2012 IEEE Conference on Computational Intelligence and Games (CIG). IEEE, 2012. p. 250-256.

[5] ALHEJALI, Atif M.; LUCAS, Simon M. Evolving diverse Ms. Pac-Man playing agents using genetic programming. In: 2010 UK Workshop on Computational Intelligence (UKCI). IEEE, 2010. p. 1-6.

[6] GNANASEKARAN, Abeynaya; FABA, Jordi Feliu; AN, Jing. Reinforcement Learning in Pacman., 2017.

[7] The Pac-Man Projects. Disponível em https://cs.brynmawr.edu/Courses/cs372/fall2017/ (acesso em junho de 2021).